### Шаг №1 - Инициализация класса

Создайте класс с именем MyLogReg. Данный класс при инициализации должен принимать на вход два параметра:

n_iter – количество шагов градиентного спуска.

По-умолчанию: 10

learning_rate – коэффициент скорости обучения градиентного спуска.

По-умолчанию: 0.1

Все переданные (или дефолтные) параметры должны быть сохранены внутри класса.

При обращении к экземпляру класса (или при передачи его в функцию print) необходимо распечатать строку по следующему шаблону:

MyLogReg class: n_iter=n_iter,

learning_rate=learning_rate

In [ ]:
class MyLogReg():

  def __init__(self, n_iter=10, learning_rate=0.1):
    self.n_iter = n_iter
    self.learning_rate = learning_rate

  def __str__(self):
    params = ', '.join(f'{key}={value}' for key, value in self.__dict__.items())
    return f'{__class__.__name__} class: {params}'

In [ ]:
print(MyLogReg())

MyLogReg class: n_iter=10, learning_rate=0.1


### Шаг №2 - Метод .fit()

Реализуем обучение нашей модели:

В инициализатор класса добавить новый параметр – weights – который будет хранить веса модели. По умолчанию он ничего не содержит.
Вам необходимо реализовать метод fit в Вашем классе. Данный метод должен делать следующее:
На вход принимать три атрибута:
- X – все фичи в виде датафрейма пандаса.
  Примечание: даже если фича будет всего одна, это все равно будет датафрейм, а не серия.
- y – целевая переменная в виде пандасовской серии.
- verbose – указывает, на какой итерации выводить лог. Например, значение 10 означает, что на каждой 10 итерации градиентного спуска будет печататься лог. Значение по умолчанию = False (т.е. ничего не выводится).
Дополнить переданную матрицу фичей единичным столбцом слева.
Определить, сколько фичей передано и создать вектор весов, состоящий из одних единиц соответствующей длинны: т.е. количество фичей + 1.
Дальше в цикле (до n_iter):
- Предсказать
y
^
y
^
​
 .
- Посчитать ошибку (LogLoss).
- Вычислить градиент.
- Сделать шаг размером learning rate в противоположную от градиента сторону.
- Сохранить обновленные веса внутри класса.
В процессе обучения необходимо выводить лог, в котором указывать номер итерации и значение функций потерь:
start | loss: 42027.65
100 | loss: 1222.87
200 | loss: 232.17
300 | loss: 202.4
где start - значении функции потерь до начала обучения. Далее выводится каждое i-ое значение итерации, переданное в параметре verbose. Если verbose = False, то лог не выводится вовсе.
Метод ничего не возвращает.
Необходимо реализовать метод get_coef, который будет возвращать значения весов в виде массива NumPy.

In [ ]:
import pandas as pd
import numpy as np

class MyLogReg():
  def __init__(self, n_iter=10, learning_rate=0.1, weights=None):
    self.n_iter = n_iter
    self.learning_rate = learning_rate
    self.weights = weights

  def __str__(self):
    params = ', '.join(f'{key}={value}' for key, value in self.__dict__.items())
    return f'{__class__.__name__} class: {params}'

  def fit(self, X:pd.DataFrame, y:pd.Series, verbose=False):
    # дополняем матрицу фичей единичным столбцом слева
    # это необходимо для перемножение первоначальных значений весов
    X = np.hstack([np.ones((X.shape[0], 1)), X.values])

    # инициализируем веса
    self.weights = np.ones(X.shape[1])

    # вычисляем начальное значение функции потерь
    for i in range(self.n_iter):
      # предсказания
      y_pred =  1 / (1 + np.exp(-X.dot(self.weights)))  # логистическая функция
      # считаем LogLoss
      log_loss = -np.mean(y*np.log(y_pred + 1e-15) + (1-y) * np.log(1 - y_pred + 1e-15))
      # вычисление градиента (частное производно)
      gradient = np.dot(X.T, (y_pred-y)) / y.size
      # обновление весов (шаг в противолодожную от градиента сторону)
      self.weights -= self.learning_rate * gradient
      if verbose and i % verbose == 0:
        print(f'{i} | loss: {log_loss}')


  def get_coef(self):
    # Возвращаем все веса, начиная с первого (пропускаем фиктивный признак)
    return self.weights[1:]

### Шаг №3 - Prediction

Научим модель выдавать предсказания. Добавьте в класс `MyLogReg` два метода `predict` и `predict_proba`. Оба метода должны делать следующее:

1. На вход принимать матрицу фичей в виде датафрейма pandas.
2. Дополнять матрицу фичей единичным вектором (первый столбец).
3. Возвращать вектор предсказаний с одним отличием:
   - `predict_proba` — возвращает вероятности (логиты, прогнанные через функцию сигмоиды).
   - `predict` — переводит вероятности в бинарные классы по порогу > 0.5.

Формула для предсказания вероятностей выглядит так:

$$
\hat{y} = \frac{1}{1 + e^{-XW}}
$$

где:

- $X$ — матрица фичей;
- $W$ — вектор весов.


In [ ]:
class MyLogReg():
  def __init__(self, n_iter=10, learning_rate=0.1, weights=None):
    self.n_iter = n_iter
    self.learning_rate = learning_rate
    self.weights = weights

  def __str__(self):
    params = ', '.join(f'{key}={value}' for key, value in self.__dict__.items())
    return f'{__class__.__name__} class: {params}'

  def fit(self, X:pd.DataFrame, y:pd.Series, verbose=False):
    # дополняем матрицу фичей единичным столбцом слева
    # это необходимо для перемножения первоначальных значений весов
    X = np.hstack([np.ones((X.shape[0], 1)), X.values])
    # инициализируем веса
    self.weights = np.ones(X.shape[1])
    # вычисляем начальное значение функции потерь
    for i in range(self.n_iter):
      # предсказания
      y_pred = 1 / (1 + np.exp(-X.dot(self.weights))) # логистическая ф-ия
      # считаем LogLoss
      log_loss = -np.mean(y * np.log(y_pred + 1e-9) + (1-y) * np.log(1 - y_pred + 1e-9))
      # вычисление градиента (частной производной функции потерь)
      gradient = np.dot(X.T, (y_pred-y)) / y.size
      # обновление весов (шаг в противоположную от градиента сторону)
      self.weights -= self.learning_rate * gradient
      if verbose and i % verbose == 0:
        print( f'{i} | loss: {log_loss}')

  def get_coef(self):
    return self.weights[1:]

  def predict_proba(self, X:pd.DataFrame):
    # дополняем матрицу фичей единичным столбцом слева
    X = np.hstack([np.ones((X.shape[0], 1)), X.values])
    return 1 / (1 + np.exp(-X.dot(self.weights)))

  def predict(self, X:pd.DataFrame):
    # дополняем матрицу фичей единичным столбцом слева
    X = np.hstack([np.ones((X.shape[0], 1)), X.values])
    return (1 / (1 + np.exp(-X.dot(self.weights))) > 0.5).astype(int)

### Шаг №4 - Метрики качества

Реализация метрик в классе MyLogReg

Теперь реализуем метрики на практике:

Добавьте в класс `MyLogReg` параметр `metric`, который будет принимать одно из следующих значений:
- `accuracy`
- `precision`
- `recall`
- `f1`
- `roc_auc`

По умолчанию: `None`.

При обучении добавьте в вывод расчет метрики:

start | loss: 42027.65 | metric_name: 234.65

100 | loss: 1222.87 | metric_name: 114.35

200 | loss: 232.17 | metric_name: 58.2

300 | loss: 202.4 | metric_name: 46.01


Если метрика не задана, то ничего дополнительно выводить не нужно.

Добавьте метод `get_best_score`, который возвращает значение метрики уже обученной модели.

Примечание

Метрика ROC AUC сильно зависит от порядка следования объектов. А разная последовательность арифметических операций может привести к тому, что скоры в последних знаках после запятой могут отличаться. Что, в свою очередь, приведет к разной сортировке объектов и разной оценке ROC AUC. Чтобы этого избежать, округлите скоры при вычислении метрики ROC AUC до 10 знака после запятой. При этом скоры, возвращаемые моделью, должны остаться в неизменном виде.

MySolution

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

class MyLogReg():
  def __init__(self, n_iter=100, learning_rate=0.1, weights=None, metric=None):
    self.n_iter = n_iter
    self.learning_rate = learning_rate
    self.weights = weights
    self.metric = metric
    self.best_score = None

  def __str__(self):
    params = ', '.join(f'{key}={value}' for key, value in self.__dict__.items())
    return f'{__class__.__name__} class: {params}'

  def compute_metric(self, y, y_pred):
    y_pred_binary = (y_pred >= 0.5).astype(int)
    # accuracy
    if self.metric == 'accuracy':
      return accuracy_score(y, y_pred_binary)
    # precision
    elif self.metric == 'precision':
      return precision_score(y, y_pred_binary)
    # recall
    elif self.metric == 'recall':
      return recall_score(y, y_pred_binary)
    # f1
    elif self.metric == 'f1':
      return f1_score(y, y_pred_binary)
    # roc_auc
    elif self.metric == 'roc_auc':
      # округляем предсказания до 10 знаков после запятой только для ROC-AUC
      y_pred_rounded = np.round(y_pred, 10)
      return roc_auc_score(y, y_pred_rounded)

  def fit(self, X:pd.DataFrame, y:pd.Series, verbose=False):
    # дополняем матрицу фичей единичным столбцом слева
    # это необходимо для перемножения первоначальных значений весов
    X = np.hstack([np.ones((X.shape[0], 1)), X.values])
    # инициализируем веса
    self.weights = np.ones(X.shape[1])
    # вычисляем начальное значение функции потерь
    for i in range(self.n_iter):
      y_pred = 1 / (1 + np.exp(-X.dot(self.weights)))
      # считаем logloss
      log_loss = -np.mean(y * np.log(y_pred + 1e-15) + (1-y) * np.log(1 - y_pred + 1e-15))
      # вычисление градиента (частной производной функции потерь)
      gradient = np.dot(X.T, (y_pred-y)) / y.size
      # обновление весов (шаг в противоположную сторону от градиента)
      self.weights -= self.learning_rate * gradient
      # Логирование на каждой итерации с отображением метрики
      if verbose and i % verbose == 0:
        metric_value = self.compute_metric(y, y_pred)
        if self.metric:
          print(f'i | loss: {loss:.2f} | {self.metric}: {metric_value:.2f}')
        else:
          print( f'{i} | loss: {log_loss}')
    # Вычисляем метрику на последнм шаге после завершения всеъ итераций
    y_pred_final = 1 / (1 + np.exp(-X.dot(self.weights)))  # Используем логистическую функцию
    self.best_score = round(self.compute_metric(y, y_pred_final), 10)

  def get_coef(self):
    return self.weights[1:]

  def predict_proba(self, X:pd.DataFrame):
    X = np.hstack([np.ones((X.shape[0],1)), X.values])
    return 1 / (1 +np.exp(-X.dot(self.weights)))

  def predict(self, X:pd.DataFrame):
    X = np.hstack([np.ones((X.shape[0],1)), X.values])
    return (1 / (1 +np.exp(-X.dot(self.weights))) > 0.5).astype(int)

  def get_best_score(self):
    return self.best_score

Чье то решение с курсов, взял, чтобы в будущем разобраться с расчетом метрик

In [ ]:
class TableMetrics:
    def __init__(self, y_true, y_pred_proba, metric):
        self.y_true = np.array(y_true)
        self.y_pred = np.array((y_pred_proba > 0.5).astype(int))
        self.y_pred_proba = y_pred_proba
        self.tp = np.sum((self.y_true == 1) & (self.y_pred == 1))
        self.fp = np.sum((self.y_true == 0) & (self.y_pred == 1))
        self.fn = np.sum((self.y_true == 1) & (self.y_pred == 0))
        self.tn = np.sum((self.y_true == 0) & (self.y_pred == 0))
        self.metric = metric

    def score(self):
        if self.metric == 'accuracy':
            return self.accuracy()
        elif self.metric == 'precision':
            return self.precision()
        elif self.metric == 'recall':
            return self.recall()
        elif self.metric == 'f1':
            return self.f1_score()
        elif self.metric == 'roc_auc':
            return self.roc_auc()
        elif self.metric == 'false_positive_rate':
            return self.false_positive_rate()
        return None

    def accuracy(self):
        if self.tp + self.tn + self.fp + self.fn == 0:
            return 0
        return (self.tp + self.tn) / (self.tp + self.tn + self.fp + self.fn)

    def precision(self):
        if self.tp + self.fp == 0:
            return 0
        return self.tp / (self.tp + self.fp)

    def recall(self):
        if self.tp + self.fn == 0:
            return 0
        return self.tp / (self.tp + self.fn)

    def false_positive_rate(self):
        return self.fp / (self.fp + self.tn)

    def f1_score(self):
        precision = self.precision()
        recall = self.recall()
        if precision + recall == 0:
            return 0
        return 2 * (precision * recall) / (precision + recall)

    def roc_auc(self):
        sqr = 0
        n_ones = np.sum(self.y_true == 1)
        n_zeroes = np.sum(self.y_true == 0)
        m = n_ones * n_zeroes
        trip = sorted(zip(self.y_pred_proba, self.y_true), reverse=True)
        for _, true in trip:
            if true == 1:
                sqr += n_zeroes
            else:
                n_zeroes -= 1
        return sqr / m

class MyLogReg:
    def __init__(self, n_iter=10, learning_rate=0.1, metric=None):
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        self.weights = None
        self.metric = metric

    def fit(self, X, y, verbose=False):
        n_samples, n_features = X.shape
        ones = np.ones((n_samples, 1))
        X = np.hstack((ones, X))

        self.weights = np.ones((n_features + 1, 1))

        EPS = 1e-15
        for iter in range(self.n_iter + 1):
            z = X @ self.weights
            y_pred = self.sigmoid(z).flatten()
            logloss = -np.mean(y * np.log(y_pred + EPS) - (1 - y) * np.log(1 - y_pred + EPS))
            grad = ((y_pred - y) @ X) / n_samples
            self.weights -= self.learning_rate * grad.reshape(-1, 1)

            self.scores = TableMetrics(y, y_pred, self.metric)

            if verbose and iter % verbose == 0:
                print(f"{iter if iter != 0 else 'start'} | loss: {logloss}", f"| {self.metric}: {self.scores.score()}" if self.metric else '')

    def predict_proba(self, X):
        n_samples, _ = X.shape
        ones = np.ones((n_samples, 1))
        X = np.hstack((ones, X))
        return self.sigmoid(X @ self.weights)

    def predict(self, X):
        proba = self.predict_proba(X)
        return (proba > 0.5).astype(int)

    def get_coef(self):
        return self.weights[1:]

    def get_best_score(self):
        self.scores.roc_auc()
        return self.scores.score()

    def sigmoid(self, z):
        z = np.clip(z, -250, 250)
        return 1 / (1 + np.exp(-z))

    def __str__(self):
        return f"MyLogReg class: n_iter={self.n_iter}, learning_rate={self.learning_rate}"

### Шаг №5 - Регуляризация

Регуляризация

Теперь повторим регуляризацию для логистической регрессии. Она ничем не отличается от линейной — просто добавляем веса к функции потерь:

L1 регуляризация (или Lasso регрессия)

Добавляем модуль суммы весов к функции потерь:

$$
LassoLogLoss = -\frac{1}{n} \sum_{i=1}^{n} \left( y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i) \right) + \lambda_1 \sum_{j=1}^{m} |w_j|
$$

Градиент:

$$
\nabla (LassoLogLoss) = \begin{bmatrix} \theta_0 \\ \theta_1 \\ \vdots \\ \theta_m \end{bmatrix} + \lambda_1 \begin{bmatrix} \text{sgn}(w_0) \\ \text{sgn}(w_1) \\ \vdots \\ \text{sgn}(w_m) \end{bmatrix}
$$

или в виде матричного перемножения:

$$
\nabla (LassoLogLoss) = \frac{1}{n} (\hat{Y} - Y) X + \lambda_1 \, \text{sgn}(W)
$$

L2 регуляризация (или Ridge регрессия)

Добавляем квадрат весов к функции потерь:

$$
RidgeLogLoss = -\frac{1}{n} \sum_{i=1}^{n} \left( y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i) \right) + \lambda_2 \sum_{j=1}^{m} w_j^2
$$

Градиент:

$$
\nabla (RidgeLogLoss) = \frac{1}{n} (\hat{Y} - Y) X + 2 \lambda_2 W
$$

ElasticNet

Добавляем и квадра и модуль весов:

$$
ElasticNetLogLoss = -\frac{1}{n} \sum_{i=1}^{n} \left( y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i) \right) + \lambda_1 \sum_{j=1}^{m} |w_j| + \lambda_2 \sum_{j=1}^{m} w_j^2
$$

Градиент:

$$
\nabla (ElasticNetLogLoss) = \frac{1}{n} (\hat{Y} - Y) X + \lambda_1 \, \text{sgn}(W) + 2 \lambda_2 W
$$

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

class MyLogReg():
  def __init__(self, n_iter=100, learning_rate=0.1, weights=None, metric=None, reg=None, l1_coef=0, l2_coef=0):
    self.n_iter = n_iter
    self.learning_rate = learning_rate
    self.weights = weights
    self.metric = metric
    self.best_score = None
    #регуляризация
    self.reg = reg
    self.l1_coef = l1_coef
    self.l2_coef = l2_coef

  def __str__(self):
    params = ', '.join(f'{key}={value}' for key, value in self.__dict__.items())
    return f'{__class__.__name__} class: {params}'

  def compute_metric(self, y, y_pred):
    y_pred_binary = (y_pred >= 0.5).astype(int)
    # accuracy
    if self.metric == 'accuracy':
      return accuracy_score(y, y_pred_binary)
    # precision
    elif self.metric == 'precision':
      return precision_score(y, y_pred_binary)
    # recall
    elif self.metric == 'recall':
      return recall_score(y, y_pred_binary)
    # f1
    elif self.metric == 'f1':
      return f1_score(y, y_pred_binary)
    # roc_auc
    elif self.metric == 'roc_auc':
      # округляем предсказания до 10 знаков после запятой только для ROC-AUC
      y_pred_rounded = np.round(y_pred, 10)
      return roc_auc_score(y, y_pred_rounded)

  def compute_log_loss(self, y, y_pred):
    # обычный log_loss
    log_loss = -np.mean(y * np.log(y_pred + 1e-15) + (1 - y) * np.log(1 - y_pred + 1e-15))
    # log_loss c L1-регуляризацией (добавляем модуль суммы весов к функции потерь)
    if self.reg == 'l1':
      log_loss += self.l1_coef * np.sum(np.abs(self.weights))
    # log_loss c L2-регуляризацией (добавляем сумму квадратов весов к функции потерь)
    elif self.reg == 'l2':
      log_loss += self.l2_coef * np.sum(self.weights ** 2)
    #Elastic net
    elif self.reg == 'elasticnet':
      log_loss += self.l1_coef * np.sum(np.abs(self.weights)) + self.l2_coef * np.sum(self.weights ** 2)

    return log_loss


  def compute_gradient(self, X, y, y_pred):
    # стоковый градиент
    gradient = np.dot(X.T, (y_pred-y)) / y.size
    # gradient с L1-регуляризацией
    if self.reg == 'l1':
      gradient += self.l1_coef * np.sign(self.weights)
    # gradient с L2-регуляризацией
    elif self.reg == 'l2':
      gradient += 2 * self.l2_coef * self.weights
    # gradient c ElasticNet
    elif self.reg == 'elasticnet':
      gradient += self.l1_coef * np.sign(self.weights) + 2 * self.l2_coef * self.weights
    return gradient


  def fit(self, X:pd.DataFrame, y:pd.Series, verbose=False):
    # дополняем матрицу фичей единичным столбцом слева
    # это необходимо для перемножения первоначальных значений весов
    X = np.hstack([np.ones((X.shape[0], 1)), X.values])
    # инициализируем веса
    self.weights = np.ones(X.shape[1])
    # вычисляем начальное значение функции потерь
    for i in range(self.n_iter):
      y_pred = 1 / (1 + np.exp(-X.dot(self.weights)))
      # считаем logloss
      log_loss = self.compute_log_loss(y, y_pred)
      # вычисление градиента (частной производной функции потерь)
      gradient = self.compute_gradient(X, y, y_pred)
      # обновление весов (шаг в противоположную сторону от градиента)
      self.weights -= self.learning_rate * gradient
      # Логирование на каждой итерации с отображением метрики
      if verbose and i % verbose == 0:
        metric_value = self.compute_metric(y, y_pred)
        if self.metric:
          print(f'i | loss: {loss:.2f} | {self.metric}: {metric_value:.2f}')
        else:
          print( f'{i} | loss: {log_loss}')
    # Вычисляем метрику на последнм шаге после завершения всеъ итераций
    y_pred_final = 1 / (1 + np.exp(-X.dot(self.weights)))  # Используем логистическую функцию
    final_metric = self.compute_metric(y, y_pred_final)
    # Добавлена проверка на None
    if final_metric is not None:
        self.best_score = round(final_metric, 10)
    else:
        self.best_score = None


  def get_coef(self):
    return self.weights[1:]

  def predict_proba(self, X:pd.DataFrame):
    X = np.hstack([np.ones((X.shape[0],1)), X.values])
    return 1 / (1 + np.exp(-X.dot(self.weights)))

  def predict(self, X:pd.DataFrame):
    X = np.hstack([np.ones((X.shape[0],1)), X.values])
    return (1 / (1 + np.exp(-X.dot(self.weights))) > 0.5).astype(int)

  def get_best_score(self):
    return self.best_score

### Шаг №6 - Градиентный спуск

Добавим оптимальный регулятор скорости обучения.

Возьмите код из предыдущего шага и модифицируйте в нем параметр learning_rate так, чтобы он принимал и число и лямбда-функцию:

Если на вход пришло число, то работаем, как и раньше.
Если на вход пришла lambda-функция, то вычисляем learning_rate на основе переданной формулы. Примерно такой:
lambda iter: 0.5 * (0.85 ** iter)
Можете дополнительно вывести значение learning_rate в лог тренировки.

З.Ы. Напоминаю, что нумерация шагов (теперь) должна быть от 1 до n_iter (включительно).

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

class MyLogReg():
  def __init__(self, n_iter=100, learning_rate=0.1, weights=None, metric=None, reg=None, l1_coef=0, l2_coef=0):
    self.n_iter = n_iter
    self.learning_rate = learning_rate
    self.weights = weights
    self.metric = metric
    self.best_score = None
    #регуляризация
    self.reg = reg
    self.l1_coef = l1_coef
    self.l2_coef = l2_coef

  def __str__(self):
    params = ', '.join(f'{key}={value}' for key, value in self.__dict__.items())
    return f'{__class__.__name__} class: {params}'

  def compute_metric(self, y, y_pred):
    y_pred_binary = (y_pred >= 0.5).astype(int)
    # accuracy
    if self.metric == 'accuracy':
      return accuracy_score(y, y_pred_binary)
    # precision
    elif self.metric == 'precision':
      return precision_score(y, y_pred_binary)
    # recall
    elif self.metric == 'recall':
      return recall_score(y, y_pred_binary)
    # f1
    elif self.metric == 'f1':
      return f1_score(y, y_pred_binary)
    # roc_auc
    elif self.metric == 'roc_auc':
      # округляем предсказания до 10 знаков после запятой только для ROC-AUC
      y_pred_rounded = np.round(y_pred, 10)
      return roc_auc_score(y, y_pred_rounded)

  def compute_log_loss(self, y, y_pred):
    # обычный log_loss
    log_loss = -np.mean(y * np.log(y_pred + 1e-15) + (1 - y) * np.log(1 - y_pred + 1e-15))
    # log_loss c L1-регуляризацией (добавляем модуль суммы весов к функции потерь)
    if self.reg == 'l1':
      log_loss += self.l1_coef * np.sum(np.abs(self.weights))
    # log_loss c L2-регуляризацией (добавляем сумму квадратов весов к функции потерь)
    elif self.reg == 'l2':
      log_loss += self.l2_coef * np.sum(self.weights ** 2)
    #Elastic net
    elif self.reg == 'elasticnet':
      log_loss += self.l1_coef * np.sum(np.abs(self.weights)) + self.l2_coef * np.sum(self.weights ** 2)

    return log_loss


  def compute_gradient(self, X, y, y_pred):
    # стоковый градиент
    gradient = np.dot(X.T, (y_pred-y)) / y.size
    # gradient с L1-регуляризацией
    if self.reg == 'l1':
      gradient += self.l1_coef * np.sign(self.weights)
    # gradient с L2-регуляризацией
    elif self.reg == 'l2':
      gradient += 2 * self.l2_coef * self.weights
    # gradient c ElasticNet
    elif self.reg == 'elasticnet':
      gradient += self.l1_coef * np.sign(self.weights) + 2 * self.l2_coef * self.weights
    return gradient


  def fit(self, X:pd.DataFrame, y:pd.Series, verbose=False):
    # дополняем матрицу фичей единичным столбцом слева
    # это необходимо для перемножения первоначальных значений весов
    X = np.hstack([np.ones((X.shape[0], 1)), X.values])
    # инициализируем веса
    self.weights = np.ones(X.shape[1])
    # вычисляем начальное значение функции потерь
    for i in range(1, self.n_iter+1):
      y_pred = 1 / (1 + np.exp(-X.dot(self.weights)))
      # считаем logloss
      log_loss = self.compute_log_loss(y, y_pred)
      # вычисление градиента (частной производной функции потерь)
      gradient = self.compute_gradient(X, y, y_pred)
      # обновление весов (шаг в противоположную сторону от градиента)
      if callable(self.learning_rate):
         lr = self.learning_rate(i)
      else:
         lr = self.learning_rate

      self.weights -= lr * gradient
      # Логирование на каждой итерации с отображением метрики
      if verbose and i % verbose == 0:
        metric_value = self.compute_metric(y, y_pred)
        if self.metric:
          print(f'i | loss: {loss:.2f} | {self.metric}: {metric_value:.2f}')
        else:
          print( f'{i} | loss: {log_loss}')
    # Вычисляем метрику на последнм шаге после завершения всеъ итераций
    y_pred_final = 1 / (1 + np.exp(-X.dot(self.weights)))  # Используем логистическую функцию
    final_metric = self.compute_metric(y, y_pred_final)
    # Добавлена проверка на None
    if final_metric is not None:
        self.best_score = round(final_metric, 10)
    else:
        self.best_score = None


  def get_coef(self):
    return self.weights[1:]

  def predict_proba(self, X:pd.DataFrame):
    X = np.hstack([np.ones((X.shape[0],1)), X.values])
    return 1 / (1 + np.exp(-X.dot(self.weights)))

  def predict(self, X:pd.DataFrame):
    X = np.hstack([np.ones((X.shape[0],1)), X.values])
    return (1 / (1 + np.exp(-X.dot(self.weights))) > 0.5).astype(int)

  def get_best_score(self):
    return self.best_score


### Шаг №7 - Стохастический градиентный спуск

- И последнее... научимся выполнять стохастический градиентный спуск

Для этого добавьте в класс `MyLogReg` два новых параметра:

- `sgd_sample` — кол-во образцов, которое будет использоваться на каждой итерации обучения. Может принимать либо целые числа, либо дробные от `0.0` до `1.0`.  
  **По-умолчанию**: `None`
- `random_state` — для воспроизводимости результата зафиксируем сид (об этом далее).  
  **По-умолчанию**: `42`

- Внесем изменения в алгоритм обучения:

1. В начале обучения фиксируем сид (см. ниже).
2. В начале каждого шага формируется новый мини-пакет на основе параметра `sgd_sample`:
   - Если заданы целые числа, то из исходного датасета берется ровно столько примеров, сколько указано.
   - Если задано дробное число, то рассматриваем его как долю от количества строк в исходном датасете (округленное до целого числа).
3. Расчет градиента (и последующее изменение весов) делаем на основе мини-пакета.
4. Все остальные параметры, если они заданы (например, регуляризация), также должны учитываться при обучении.
5. Ошибку и метрику необходимо считать на всем датасете, а не на мини-пакете.
6. Если `sgd_sample = None`, то обучение выполняется как раньше (на всех данных).

- Случайная генерация

Случайные подвыборки будем генерировать, как и раньше.

- В начале обучения посредством модуля `random` фиксируем сид:

    ```python
    random.seed(<random_state>)
    ```

- В начале каждой итерации сформируем номера строк, которые стоит отобрать:

    ```python
    sample_rows_idx = random.sample(range(X.shape[0]), <sgd_sample>)
    ```

**З.Ы.** Модуль `random` уже импортирован.


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

class MyLogReg():
  def __init__(self, n_iter=100, learning_rate=0.1, weights=None, metric=None, reg=None, l1_coef=0, l2_coef=0, sgd_sample=None, random_state=42):
    self.n_iter = n_iter
    self.learning_rate = learning_rate
    self.weights = weights
    self.metric = metric
    self.best_score = None
    #регуляризация
    self.reg = reg
    self.l1_coef = l1_coef
    self.l2_coef = l2_coef
    # stochastic_grad
    self.sgd_sample = sgd_sample
    self.random_state = random_state

  def __str__(self):
    params = ', '.join(f'{key}={value}' for key, value in self.__dict__.items())
    return f'{__class__.__name__} class: {params}'

  def compute_metric(self, y, y_pred):
    y_pred_binary = (y_pred >= 0.5).astype(int)
    # accuracy
    if self.metric == 'accuracy':
      return accuracy_score(y, y_pred_binary)
    # precision
    elif self.metric == 'precision':
      return precision_score(y, y_pred_binary)
    # recall
    elif self.metric == 'recall':
      return recall_score(y, y_pred_binary)
    # f1
    elif self.metric == 'f1':
      return f1_score(y, y_pred_binary)
    # roc_auc
    elif self.metric == 'roc_auc':
      # округляем предсказания до 10 знаков после запятой только для ROC-AUC
      y_pred_rounded = np.round(y_pred, 10)
      return roc_auc_score(y, y_pred_rounded)

  def compute_log_loss(self, y, y_pred):
    # обычный log_loss
    log_loss = -np.mean(y * np.log(y_pred + 1e-15) + (1 - y) * np.log(1 - y_pred + 1e-15))
    # log_loss c L1-регуляризацией (добавляем модуль суммы весов к функции потерь)
    if self.reg == 'l1':
      log_loss += self.l1_coef * np.sum(np.abs(self.weights))
    # log_loss c L2-регуляризацией (добавляем сумму квадратов весов к функции потерь)
    elif self.reg == 'l2':
      log_loss += self.l2_coef * np.sum(self.weights ** 2)
    #Elastic net
    elif self.reg == 'elasticnet':
      log_loss += self.l1_coef * np.sum(np.abs(self.weights)) + self.l2_coef * np.sum(self.weights ** 2)

    return log_loss


  def compute_gradient(self, X, y, y_pred):
    # стоковый градиент
    gradient = np.dot(X.T, (y_pred-y)) / y.size
    # gradient с L1-регуляризацией
    if self.reg == 'l1':
      gradient += self.l1_coef * np.sign(self.weights)
    # gradient с L2-регуляризацией
    elif self.reg == 'l2':
      gradient += 2 * self.l2_coef * self.weights
    # gradient c ElasticNet
    elif self.reg == 'elasticnet':
      gradient += self.l1_coef * np.sign(self.weights) + 2 * self.l2_coef * self.weights
    return gradient


  def fit(self, X:pd.DataFrame, y:pd.Series, verbose=False):
    # фиксируем сид для воспроизводимости
    random.seed(self.random_state)
    # дополняем матрицу фичей единичным столбцом слева
    # это необходимо для перемножения первоначальных значений весов
    X = np.hstack([np.ones((X.shape[0], 1)), X.values])
    # инициализируем веса
    self.weights = np.ones(X.shape[1])
    # Определяем кол-во примеров для мини-батча
    if self.sgd_sample is not None:
      if isinstance(self.sgd_sample, float) and 0 < self.sgd_sample <= 1:
        batch_size = int(self.sgd_sample * X.shape[0]) # Интерпретируем как долю от данных
      elif isinstance(self.sgd_sample, int) and self.sgd_sample > 0:
        batch_size = self.sgd_sample # Определенное кол-во строк
      else:
        raise ValueError('sgd_sample должен быть целым числом или дробным значением от 0 до 1.')
    else:
      batch_size = X.shape[0] # Если не задано, то берем все данные
    # вычисляем начальное значение функции потерь
    for i in range(1, self.n_iter+1):
      # формируем мини-батч
      if batch_size < X.shape[0]:
        sample_rows_idx = random.sample(range(X.shape[0]), batch_size)
        X_batch = X[sample_rows_idx]
        y_batch = y.values[sample_rows_idx]
      else:
        X_batch = X
        y_batch = y.values
      # Предсказания для мини-батча
      y_pred = 1 / (1 + np.exp(-X_batch.dot(self.weights)))
      # считаем logloss
      log_loss = self.compute_log_loss(y_batch, y_pred)
      # вычисление градиента (частной производной функции потерь)
      gradient = self.compute_gradient(X_batch, y_batch, y_pred)
      # обновление весов (шаг в противоположную сторону от градиента)
      if callable(self.learning_rate):
         lr = self.learning_rate(i)
      else:
         lr = self.learning_rate

      self.weights -= lr * gradient
      # Логирование на каждой итерации с отображением метрики
      if verbose and i % verbose == 0:
        metric_value = self.compute_metric(y, y_pred)
        if self.metric:
          print(f'i | loss: {loss:.2f} | {self.metric}: {metric_value:.2f}')
        else:
          print( f'{i} | loss: {log_loss}')
    # Вычисляем метрику на последнм шаге после завершения всеъ итераций
    y_pred_final = 1 / (1 + np.exp(-X.dot(self.weights)))  # Используем логистическую функцию
    final_metric = self.compute_metric(y, y_pred_final)
    # Добавлена проверка на None
    if final_metric is not None:
        self.best_score = round(final_metric, 10)
    else:
        self.best_score = None


  def get_coef(self):
    return self.weights[1:]

  def predict_proba(self, X:pd.DataFrame):
    X = np.hstack([np.ones((X.shape[0],1)), X.values])
    return 1 / (1 + np.exp(-X.dot(self.weights)))

  def predict(self, X:pd.DataFrame):
    X = np.hstack([np.ones((X.shape[0],1)), X.values])
    return (1 / (1 + np.exp(-X.dot(self.weights))) > 0.5).astype(int)

  def get_best_score(self):
    return self.best_score